In [1]:
import pandas as pd
import datetime

def process_csv(filepath, product_index):
    """
    This function takes the file path and product index as inputs and returns the filtered trade data dataframe
    for a particular product.
    
    Parameters:
    filepath (str) : file path for the trade data file
    product_index (int) : index of the product in the list of all products
    
    Returns:
    df_product_filtered (pandas dataframe) : dataframe for the filtered product
    """
    
    # Read the CSV file and convert the "TIME_PERIOD" column to datetime
    trade_data = pd.read_csv(filepath)
    trade_data["TIME_PERIOD"] = pd.to_datetime(trade_data["TIME_PERIOD"], format = '%Y-%m').dt.date
    
    # Rename columns and remove duplicates
    trade_data = trade_data.rename(columns={"partner": "PARTNER_codes", "declarant": "DECLARANT_codes"})
    trade_data = trade_data.drop_duplicates()
    
    # Get the list of all products
    products = trade_data["product"].drop_duplicates().to_list()
    
    # Filter the data for a particular product
    product_filter = products[product_index]
    
    # Read the partner countries data
    partner_countries = pd.read_csv("trade_data/partners.csv", sep = ";")
    
    # Merge the trade data and partner country data on the "PARTNER_codes" column
    trade_data = pd.merge(trade_data, partner_countries, on = ['PARTNER_codes'], how = 'inner')
    
    # Pivot the trade data to create a multi-index dataframe
    trade_data_pivot = trade_data.pivot(index=['PARTNER_Labels', 'TIME_PERIOD', 'product'], 
                                  columns='indicators', 
                                  values=['OBS_VALUE'])
    
    # Flatten the column names
    trade_data_pivot.columns = trade_data_pivot.columns.map('_'.join)
    
    # Reset the index and remove missing values
    trade_data_pivot = trade_data_pivot.reset_index()
    trade_data_pivot = trade_data_pivot.dropna()
    
    # Filter the data for the selected product
    df = trade_data_pivot
    df_product_filtered = df[df['product'] == product_filter]

    
    return df_product_filtered

#Declare DataFrames for all good/productcode combinations
df_sunflower_oil_0 = process_csv("trade_data/sunflower_oil.csv", 0)
df_sunflower_oil_0['price'] = df_sunflower_oil_0['OBS_VALUE_VALUE_1000EURO'] / df_sunflower_oil_0['OBS_VALUE_QUANTITY_TON']
df_sunflower_oil_1 = process_csv("trade_data/sunflower_oil.csv", 1)
df_sunflower_oil_1['price'] = df_sunflower_oil_1['OBS_VALUE_VALUE_1000EURO'] / df_sunflower_oil_1['OBS_VALUE_QUANTITY_TON']

In [2]:
df_sunflower = df_sunflower_oil_0.merge(df_sunflower_oil_1, on=["PARTNER_Labels", "TIME_PERIOD"], how='outer')
df_sunflower = df_sunflower.fillna(0)


## AGGREGATING BOTH PRODUCT CATEGORIES

In [3]:
df_sunflower['OBS_VALUE_QUANTITY_TON'] = df_sunflower['OBS_VALUE_QUANTITY_TON_x'] + df_sunflower['OBS_VALUE_QUANTITY_TON_y']
df_sunflower['OBS_VALUE_VALUE_1000EURO'] = df_sunflower['OBS_VALUE_VALUE_1000EURO_x'] + df_sunflower['OBS_VALUE_VALUE_1000EURO_y']
df_sunflower['price'] = df_sunflower['OBS_VALUE_VALUE_1000EURO'] / df_sunflower['OBS_VALUE_QUANTITY_TON']

df_sunflower['price'].max()

inf

## CLEANING EXTREMES

In [4]:
import numpy as np

# Delete all inf and -inf
df_sunflower = df_sunflower.replace([np.inf, -np.inf], np.nan)\

# Define a function to replace extreme high values with NaN
def replace_outliers(df, column):
    z = np.abs((df[column] - df[column].mean()) / df[column].std()) # Calculate z-score for the column
    outliers = df[z > 3][column] # Identify extreme high values with a z-score greater than 3
    median = df[column].median() # Calculate the median of the column
    df.loc[z > 3, column] = np.nan # Replace extreme high values with NaN

# Define a list of columns to perform outlier treatment on
columns = ['price_x','price_y', 'price']

# Iterate over the columns and call the replace_outliers function for each one
for column in columns:
    replace_outliers(df_sunflower, column)
    

In [14]:
df_sunflower

,PARTNER_Labels,TIME_PERIOD,product_x,OBS_VALUE_QUANTITY_TON_x,OBS_VALUE_VALUE_1000EURO_x,price_x,product_y,OBS_VALUE_QUANTITY_TON_y,OBS_VALUE_VALUE_1000EURO_y,price_y,OBS_VALUE_QUANTITY_TON,OBS_VALUE_VALUE_1000EURO,price
0,Argentina,2005-01-01,15121191.0,21086.3,10489.97,0.497478,0.0,0.0,0.00,0.000000,21086.3,10489.97,0.497478
1,Argentina,2005-02-01,15121191.0,18325.9,8946.33,0.488180,0.0,0.0,0.00,0.000000,18325.9,8946.33,0.488180
2,Argentina,2005-03-01,15121191.0,25661.5,12741.53,0.496523,0.0,0.0,0.00,0.000000,25661.5,12741.53,0.496523
3,Argentina,2005-04-01,15121191.0,33839.2,16665.38,0.492487,0.0,0.0,0.00,0.000000,33839.2,16665.38,0.492487
4,Argentina,2005-05-01,15121191.0,41051.1,19921.35,0.485282,0.0,0.0,0.00,0.000000,41051.1,19921.35,0.485282
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4479,United States,2020-05-01,0.0,0.0,0.00,0.000000,15121990.0,0.1,0.06,0.600000,0.1,0.06,0.600000
4480,United States,2020-07-01,0.0,0.0,0.00,0.000000,15121990.0,3.9,4.02,1.030769,3.9,4.02,1.030769
4481,United States,2021-01-01,0.0,0.0,0.00,0.000000,15121990.0,0.1,0.07,0.700000,0.1,0.07,0.700000
4482,United States,2021-02-01,0.0,0.0,0.00,0.000000,15121990.0,1.9,3.15,1.657895,1.9,3.15,1.657895


## WEATHER DATA

In [15]:
import pandas as pd
import datetime

#Clean import oil_price_index_2015_oecd

# Read the data from the weather file
df_global_temperature = pd.read_csv('weather_data/global_temperature_data.csv')

# Rename the 'YearMonth' column to 'TIME_PERIOD'
df_global_temperature.rename(columns={'yearmonth': 'TIME_PERIOD', 'partner_labels': 'PARTNER_Labels'}, inplace=True)

# Convert the TIME_PERIOD column to datetime format
df_global_temperature['TIME_PERIOD'] = pd.to_datetime(df_global_temperature['TIME_PERIOD'], format='%Y%m')

# Convert the TIME_PERIOD column to the desired format
df_global_temperature['TIME_PERIOD'] = df_global_temperature['TIME_PERIOD'].dt.strftime('%Y-%m-%d')

# Drop columns
df_global_temperature.drop(columns=['partner_code', 'weather_countrycode', 'weather_countryname', 'iso_countrycode'], inplace=True)

df_global_temperature['TIME_PERIOD'] = pd.to_datetime(df_global_temperature['TIME_PERIOD'])
df_global_temperature

,PARTNER_Labels,TIME_PERIOD,AVG_TAVG,MIN_TMIN,MAX_TMAX
0,United Arab Emirates,2005-01-01,186.096154,110.000000,280.000000
1,United Arab Emirates,2005-02-01,204.750000,106.000000,306.500000
2,United Arab Emirates,2005-03-01,238.226667,124.500000,363.666667
3,United Arab Emirates,2005-04-01,280.984568,155.000000,423.333333
4,United Arab Emirates,2005-05-01,321.296296,192.000000,435.500000
...,...,...,...,...,...
21085,Viet Nam,2022-08-01,287.060676,244.363636,348.666667
21086,Viet Nam,2022-09-01,279.751352,233.600000,350.000000
21087,Viet Nam,2022-10-01,246.000000,238.000000,260.000000
21088,Viet Nam,2022-11-01,272.000000,252.000000,304.000000


## OIL DATA

In [16]:
import pandas as pd
import datetime

#Clean import oil_price_index_2015_oecd

# Read the data from the 'oil_price_index_2015_oecd.xlsx' file
df_oil_index = pd.read_excel('oil_prices/oil_price_index_2015_oecd.xlsx', skiprows=[0,1,2,3,4, 9012, 9013, 9014, 9015])
# Forward fill the 'Country' column
df_oil_index['Country'].fillna(method='ffill', inplace=True)
# Rename columns
df_oil_index.rename(columns={'Unnamed: 3': 'CPI_ENERGY'}, inplace=True)
df_oil_index.rename(columns={'Unnamed: 4': 'CPI_ENERGY_HARMONISED'}, inplace=True)
# Delete the 'Unnamed: 2' column
df_oil_index.drop(columns=['Unnamed: 2'], inplace=True)
# Convert the 'Time' column to datetime format
df_oil_index['Time'] = pd.to_datetime(df_oil_index['Time'], format='%b-%Y')
df_oil_index.rename(columns={'Time': 'TIME_INDEX'}, inplace=True)

#Clean import CMO-Historical-Data-Monthly.xlsx

df_commodities_price = pd.read_excel('oil_prices/CMO-Historical-Data-Monthly.xlsx', sheet_name = 'Monthly Prices', skiprows=[0,1,2,3,4,5])
# Rename columns
df_commodities_price.rename(columns={'Unnamed: 0': 'TIME_PERIOD'}, inplace=True)
df_commodities_price.rename(columns={'CRUDE_PETRO': 'OIL_PRICE $/bbl'}, inplace=True)
df_commodities_price['TIME_PERIOD'] = pd.to_datetime(df_commodities_price['TIME_PERIOD'], format='%YM%m')

df_oil_price_full = pd.DataFrame(df_commodities_price, columns=['TIME_PERIOD', 'OIL_PRICE $/bbl'])

df_oil_price = df_oil_price_full.loc[(df_oil_price_full['TIME_PERIOD'] >= '2005-01-01') & (df_oil_price_full['TIME_PERIOD'] <= '2022-10-01')]
df_oil_price

,TIME_PERIOD,OIL_PRICE $/bbl
540,2005-01-01,42.972278
541,2005-02-01,44.818211
542,2005-03-01,50.942879
543,2005-04-01,50.640476
544,2005-05-01,47.826573
...,...,...
749,2022-06-01,116.800000
750,2022-07-01,105.083333
751,2022-08-01,95.973333
752,2022-09-01,88.220000


## FUTURES PRICES (SUNFLOWER OIL)

In [54]:
# Extracted from https://www.investing.com/commodities/high-oil-content-sunflower-seed-historical-data
import pandas as pd
import datetime

#Clean import High Oil Content Sunflower Seed Futures Historical Data.csv

# Read the data from the file
df_futures_sunflower_seed = pd.read_csv('futures_data/High Oil Content Sunflower Seed Futures Historical Data.csv')

# Rename the 'date' column to 'TIME_PERIOD'
df_futures_sunflower_seed.rename(columns={'Date': 'TIME_PERIOD', 'Price': 'Sunflower Seed Price Futures'}, inplace=True)

df_futures_sunflower_seed['TIME_PERIOD'] = pd.to_datetime(df_futures_sunflower_seed['TIME_PERIOD'], format='%d/%m/%Y')
df_futures_sunflower_seed['TIME_PERIOD'] = df_futures_sunflower_seed['TIME_PERIOD'].dt.strftime('%Y/%m/%d')

df_futures_sunflower_seed.drop(columns=['Open','High','Low','Vol.','Change %'], inplace=True)

df_futures_sunflower_seed.sort_values(by='TIME_PERIOD', ascending=True, inplace=True)

# Replace commas with dots in 'Sunflower Seed Price Futures' column
df_futures_sunflower_seed['Sunflower Seed Price Futures'] = df_futures_sunflower_seed['Sunflower Seed Price Futures'].str.replace(',', '.')

# Convert 'Sunflower Seed Price Futures' column to float datatype
df_futures_sunflower_seed['Sunflower Seed Price Futures'] = df_futures_sunflower_seed['Sunflower Seed Price Futures'].astype(float)


df_futures_sunflower_seed

,TIME_PERIOD,Sunflower Seed Price Futures
138,2006/01/10,49.2
137,2006/01/11,52.0
136,2006/01/12,54.5
135,2007/01/01,54.5
134,2007/01/02,54.5
...,...,...
4,2022/01/06,267.0
3,2022/01/07,262.0
2,2022/01/08,265.0
1,2022/01/09,267.0


## MACROECONOMIC DATA

In [19]:
#import datetime
import numpy
import openpyxl
#import data
path_file_1_p = "macroeconomic_data/Population_projections.xlsx"
population_data = pd.read_excel(path_file_1_p)

path_file_1 = "macroeconomic_data/macro_economic_indicators.csv"
economic_indicators = pd.read_csv(path_file_1, sep = ";", decimal=".")

#rename column for merge later
population_data = population_data.rename(columns = {'Country Name': 'PARTNER_Labels'})
economic_indicators = economic_indicators.rename(columns = {'country_name': 'PARTNER_Labels' ,'Time':'TIME_PERIOD' })
#this is a quick fix
macro_economic_indicators = economic_indicators
# Convert year to datetime with year and month
macro_economic_indicators['year'] = pd.to_datetime(macro_economic_indicators['year'], format='%Y').dt.to_period('M')
# Set the date as the index
macro_economic_indicators.set_index('year', inplace=True)

#make df for monthly data
monthly_data = macro_economic_indicators.iloc[0:0].copy()


number_of_col = macro_economic_indicators.shape[1]
nan_row = numpy.empty((number_of_col,))
nan_row[:] = numpy.nan

#add na rows for 11 months in a year
n_row = macro_economic_indicators.shape[0] 
for i in range(n_row):
    print(i)
    holder_for_rows = macro_economic_indicators.iloc[i,:]
    current_country = macro_economic_indicators.iloc[i,0] 
    current_country_code = macro_economic_indicators.iloc[i,1] 
    for j in range(12):
        index_3 = 12 * i + j
        if j == 0 :
            monthly_data.loc[index_3] = holder_for_rows   
        else:
            monthly_data.loc[index_3] = nan_row
            monthly_data.iloc[index_3,0] = current_country
            monthly_data.iloc[index_3,1] = current_country_code

#slects als the column with numbers
c = monthly_data.columns[3:12]
monthly_data[c] = monthly_data[c].apply(pd.to_numeric)

n_countries = len(population_data['Country Code'].unique())
#fills in the date column
for i in range(n_countries-1):
    index_1 = 0   + 204*i
    index_2 = 204 + 204*i
    monthly_data.iloc[index_1:index_2,:] = monthly_data.iloc[index_1:index_2,:].interpolate().copy()
    #adding dates 
    monthly_data.iloc[index_1:index_2,2] = pd.period_range("2005-01", "2021-12", freq='M')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061


3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700


In [20]:
import pandas as pd
import datetime
import numpy as np
import openpyxl
#import data
path_file_1_p = "macroeconomic_data/Population_projections.xlsx"
population_data = pd.read_excel(path_file_1_p)

#delete the last 5 empty rows 
num_rows = len(population_data)
population_data = population_data.drop(population_data.index[num_rows-5:num_rows])
#rename a column 
population_data = population_data.rename(columns = {'Country Name': 'PARTNER_Labels','Time':'TIME_PERIOD'})
population_data = population_data.rename(columns = {'Population, total [SP.POP.TOTL]': 'A'})

#get's the index s of the columns with string
df = population_data
string_indices = df['A'][df['A'].apply(lambda x: isinstance(x, str))].index


#gets th countries corresponging to the index s 
lll = len(string_indices)
LIST_of_del_countries = [None] * lll

counter = 0 
for i in range(lll):
    LIST_of_del_countries[counter] = population_data.iloc[(string_indices[i]),0]
    counter = counter +1

list1 = list(set(LIST_of_del_countries))

print(df.shape)
print(list1)
for j in range(4000):
  
    if df.iloc[j,0] in list1:
        df = df.drop(index=[j])
        print(df.shape)

df = df.replace('..', 0)
print(df)
# boolean indexing to select rows to delete
idx = df['PARTNER_Labels'].str.contains('IDA blend')

# drop rows using boolean indexing
df = df.drop(df[idx].index)

idx = df['PARTNER_Labels'].str.contains('IDA only')
df = df.drop(df[idx].index)

idx = df['PARTNER_Labels'].str.contains('IDA total')
df = df.drop(df[idx].index)

for name in list1:
    idx = df['PARTNER_Labels'].str.contains(name)
    df = df.drop(df[idx].index)


testing= df.copy()
excel_test = pd.ExcelWriter("population_data.xlsx")
testing.to_excel(excel_test)
excel_test.save()
#make the 
monthly_data_p = population_data.iloc[0:0].copy()

number_of_col = df.shape[1]
nan_row = np.empty((number_of_col,))
nan_row[:] = np.nan

n_row = df.shape[0] 
for i in range(n_row):
    print(i)
    holder_for_rows = df.iloc[i,:]
    current_country = df.iloc[i,0]
    current_country_code = df.iloc[i,1] 
    for j in range(12):
        index_3 = 12 * i + j
        if j == 0 :
            monthly_data_p.loc[index_3] = holder_for_rows   
        else:
            monthly_data_p.loc[index_3] = nan_row
            monthly_data_p.iloc[index_3,0] = current_country
            monthly_data_p.iloc[index_3,1] = current_country_code 


# testing= monthly_data_p.copy()
# excel_test = pd.ExcelWriter("dopper2.xlsx")
# testing.to_excel(excel_test)
# excel_test.save()

c_p = population_data.columns[4]
monthly_data_p[c_p] = monthly_data_p[c_p].apply(pd.to_numeric)
n_countries = len(df['Country Code'].unique())
#fills in the date column
for i in range(n_countries-10):
    index_1 = 0   + 228*i
    index_2 = 228 + 228*i
    monthly_data_p.iloc[index_1:index_2,4] = monthly_data_p.iloc[index_1:index_2,4].interpolate().copy()
    #adding dates 
    monthly_data_p.iloc[index_1:index_2,2] = pd.period_range("2005-01", "2023-12", freq='M')

# testing= monthly_data_p.copy()
# excel_test = pd.ExcelWriter("dopper3.xlsx")
# testing.to_excel(excel_test)
# excel_test.save()


(5054, 5)
['IDA only', 'Not classified', 'Bermuda', 'Sint Maarten (Dutch part)', 'St. Martin (French part)', 'Greenland', 'IDA total', 'IDA blend', 'IBRD only']
(5053, 5)
(5052, 5)
(5051, 5)
(5050, 5)
(5049, 5)
(5048, 5)
(5047, 5)
(5046, 5)
(5045, 5)
(5044, 5)
(5043, 5)
(5042, 5)
(5041, 5)
(5040, 5)
(5039, 5)
(5038, 5)
(5037, 5)
(5036, 5)
(5035, 5)
(5034, 5)
(5033, 5)
(5032, 5)
(5031, 5)
(5030, 5)
(5029, 5)
(5028, 5)
(5027, 5)
(5026, 5)
(5025, 5)
(5024, 5)
(5023, 5)
(5022, 5)
(5021, 5)
(5020, 5)
(5019, 5)
(5018, 5)
(5017, 5)
(5016, 5)
(5015, 5)
(5014, 5)
(5013, 5)
(5012, 5)
(5011, 5)
(5010, 5)
(5009, 5)
(5008, 5)
(5007, 5)
(5006, 5)
(5005, 5)
(5004, 5)
(5003, 5)
(5002, 5)
(5001, 5)
(5000, 5)
(4999, 5)
(4998, 5)
(4997, 5)
(4996, 5)
(4995, 5)
(4994, 5)
(4993, 5)
(4992, 5)
(4991, 5)
(4990, 5)
(4989, 5)
(4988, 5)
(4987, 5)
(4986, 5)
(4985, 5)
(4984, 5)
(4983, 5)
(4982, 5)
(4981, 5)
(4980, 5)
(4979, 5)
(4978, 5)
(4977, 5)
(4976, 5)
(4975, 5)
     PARTNER_Labels Country Code  TIME_PERIOD Tim

C:\Users\basbe\AppData\Local\Temp\ipykernel_12132\1213532793.py:55: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  idx = df['PARTNER_Labels'].str.contains(name)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062


3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705


In [22]:
monthly_data_p.head(214)

,PARTNER_Labels,Country Code,TIME_PERIOD,Time Code,A
0,Afghanistan,AFG,2005-01,YR2005,2.441119e+07
1,Afghanistan,AFG,2005-02,NaN,2.449717e+07
2,Afghanistan,AFG,2005-03,NaN,2.458315e+07
3,Afghanistan,AFG,2005-04,NaN,2.466913e+07
4,Afghanistan,AFG,2005-05,NaN,2.475511e+07
...,...,...,...,...,...
209,Afghanistan,AFG,2022-06,NaN,4.159172e+07
210,Afghanistan,AFG,2022-07,NaN,4.168431e+07
211,Afghanistan,AFG,2022-08,NaN,4.177690e+07
212,Afghanistan,AFG,2022-09,NaN,4.186949e+07


In [25]:
monthly_data_p  = monthly_data_p.rename(columns = {'Country Code': 'country_code'})
# convert Period and float values to string format, and then to datetime
monthly_data_p ['TIME_PERIOD'] = monthly_data_p ['TIME_PERIOD'].apply(lambda x: str(x) if isinstance(x, pd.Period) else x)
monthly_data_p ['TIME_PERIOD'] = pd.to_datetime(monthly_data_p ['TIME_PERIOD'], errors='coerce')
monthly_data_p.head(214)

,PARTNER_Labels,country_code,TIME_PERIOD,Time Code,A
0,Afghanistan,AFG,2005-01-01,YR2005,2.441119e+07
1,Afghanistan,AFG,2005-02-01,NaN,2.449717e+07
2,Afghanistan,AFG,2005-03-01,NaN,2.458315e+07
3,Afghanistan,AFG,2005-04-01,NaN,2.466913e+07
4,Afghanistan,AFG,2005-05-01,NaN,2.475511e+07
...,...,...,...,...,...
209,Afghanistan,AFG,2022-06-01,NaN,4.159172e+07
210,Afghanistan,AFG,2022-07-01,NaN,4.168431e+07
211,Afghanistan,AFG,2022-08-01,NaN,4.177690e+07
212,Afghanistan,AFG,2022-09-01,NaN,4.186949e+07


In [26]:
monthly_data  = monthly_data.rename(columns = {'timecode': 'TIME_PERIOD'})
# convert Period and float values to string format, and then to datetime
monthly_data['TIME_PERIOD'] = monthly_data['TIME_PERIOD'].apply(lambda x: str(x) if isinstance(x, pd.Period) else x)
monthly_data['TIME_PERIOD'] = pd.to_datetime(monthly_data['TIME_PERIOD'], errors='coerce')

monthly_data.head(206)

,PARTNER_Labels,country_code,TIME_PERIOD,gdp_current_us,gdp_constant_2015_us,population_total,unemployment_total,agricultural_land,renewable_energy_consumption_perc_of_total,energy_use_kg_of_oil_equivalent_per_capita,fossil_fuel_energy_consumption_perc_of_total
year,,,,,,,,,,,
0,Argentina,ARG,2005-01-01,1.987371e+11,4.352966e+11,39070501.0,11.510000,1.299209e+06,9.640000,1712.850000,89.050000
1,Argentina,ARG,2005-02-01,2.015554e+11,4.382157e+11,39104363.5,11.390833,1.299391e+06,9.601667,1723.801667,89.016667
2,Argentina,ARG,2005-03-01,2.043738e+11,4.411348e+11,39138226.0,11.271667,1.299572e+06,9.563333,1734.753333,88.983333
3,Argentina,ARG,2005-04-01,2.071921e+11,4.440538e+11,39172088.5,11.152500,1.299754e+06,9.525000,1745.705000,88.950000
4,Argentina,ARG,2005-05-01,2.100105e+11,4.469729e+11,39205951.0,11.033333,1.299936e+06,9.486667,1756.656667,88.916667
...,...,...,...,...,...,...,...,...,...,...,...
201,Argentina,ARG,2021-10-01,4.872273e+11,5.681426e+11,45808747.0,8.740000,1.083818e+06,10.740000,2029.920000,87.720000
202,Argentina,ARG,2021-11-01,4.872273e+11,5.681426e+11,45808747.0,8.740000,1.083818e+06,10.740000,2029.920000,87.720000
203,Argentina,ARG,2021-12-01,4.872273e+11,5.681426e+11,45808747.0,8.740000,1.083818e+06,10.740000,2029.920000,87.720000


In [27]:
merged_monthly_data_df = monthly_data.merge(monthly_data_p, on=['country_code', 'TIME_PERIOD'], how='right')
merged_monthly_data_df = merged_monthly_data_df.drop('PARTNER_Labels_y', axis=1)
merged_monthly_data_df = merged_monthly_data_df.rename(columns={'PARTNER_Labels_x': 'PARTNER_Labels', 'A': 'population_projections'})
merged_monthly_data_df

,PARTNER_Labels,country_code,TIME_PERIOD,gdp_current_us,gdp_constant_2015_us,population_total,unemployment_total,agricultural_land,renewable_energy_consumption_perc_of_total,energy_use_kg_of_oil_equivalent_per_capita,fossil_fuel_energy_consumption_perc_of_total,Time Code,population_projections
0,Afghanistan,AFG,2005-01-01,6.226199e+09,9.275174e+09,2.441119e+07,NaN,379100.0,33.880000,NaN,NaN,YR2005,2.441119e+07
1,Afghanistan,AFG,2005-02-01,6.288298e+09,9.316583e+09,2.449717e+07,NaN,379100.0,33.714167,NaN,NaN,NaN,2.449717e+07
2,Afghanistan,AFG,2005-03-01,6.350396e+09,9.357992e+09,2.458315e+07,NaN,379100.0,33.548333,NaN,NaN,NaN,2.458315e+07
3,Afghanistan,AFG,2005-04-01,6.412494e+09,9.399401e+09,2.466913e+07,NaN,379100.0,33.382500,NaN,NaN,NaN,2.466913e+07
4,Afghanistan,AFG,2005-05-01,6.474593e+09,9.440810e+09,2.475511e+07,NaN,379100.0,33.216667,NaN,NaN,NaN,2.475511e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58219,NaN,ZWE,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58220,NaN,ZWE,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58221,NaN,ZWE,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58222,NaN,ZWE,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
df_macroeconomic = merged_monthly_data_df
df_macroeconomic.head(214)

,PARTNER_Labels,country_code,TIME_PERIOD,gdp_current_us,gdp_constant_2015_us,population_total,unemployment_total,agricultural_land,renewable_energy_consumption_perc_of_total,energy_use_kg_of_oil_equivalent_per_capita,fossil_fuel_energy_consumption_perc_of_total,Time Code,population_projections
0,Afghanistan,AFG,2005-01-01,6.226199e+09,9.275174e+09,2.441119e+07,NaN,379100.0,33.880000,NaN,NaN,YR2005,2.441119e+07
1,Afghanistan,AFG,2005-02-01,6.288298e+09,9.316583e+09,2.449717e+07,NaN,379100.0,33.714167,NaN,NaN,NaN,2.449717e+07
2,Afghanistan,AFG,2005-03-01,6.350396e+09,9.357992e+09,2.458315e+07,NaN,379100.0,33.548333,NaN,NaN,NaN,2.458315e+07
3,Afghanistan,AFG,2005-04-01,6.412494e+09,9.399401e+09,2.466913e+07,NaN,379100.0,33.382500,NaN,NaN,NaN,2.466913e+07
4,Afghanistan,AFG,2005-05-01,6.474593e+09,9.440810e+09,2.475511e+07,NaN,379100.0,33.216667,NaN,NaN,NaN,2.475511e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,NaN,AFG,2022-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.159172e+07
210,NaN,AFG,2022-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.168431e+07
211,NaN,AFG,2022-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.177690e+07
212,NaN,AFG,2022-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.186949e+07


## MERGING DATA

### Merged trade data, oil price

In [55]:
df_sunflower['TIME_PERIOD'] = pd.to_datetime(df_sunflower['TIME_PERIOD'])
df_merged_trade_oil = df_oil_price.merge(df_sunflower, on='TIME_PERIOD', how='outer')
df_merged_trade_oil.head(30)

,TIME_PERIOD,OIL_PRICE $/bbl,PARTNER_Labels,product_x,OBS_VALUE_QUANTITY_TON_x,OBS_VALUE_VALUE_1000EURO_x,price_x,product_y,OBS_VALUE_QUANTITY_TON_y,OBS_VALUE_VALUE_1000EURO_y,price_y,OBS_VALUE_QUANTITY_TON,OBS_VALUE_VALUE_1000EURO,price
0,2005-01-01,42.972278,Argentina,15121191.0,21086.3,10489.97,0.497478,0.0,0.0,0.00,0.000000,21086.3,10489.97,0.497478
1,2005-01-01,42.972278,Belgium,15121191.0,489.1,414.24,0.846943,15121990.0,1171.7,839.46,0.716446,1660.8,1253.70,0.754877
2,2005-01-01,42.972278,France,15121191.0,1295.6,886.20,0.684007,15121990.0,1631.3,1024.46,0.628002,2926.9,1910.66,0.652793
3,2005-01-01,42.972278,Germany,15121191.0,81.0,79.89,0.986296,15121990.0,1228.8,939.09,0.764233,1309.8,1018.98,0.777966
4,2005-01-01,42.972278,Ireland,15121191.0,246.0,192.04,0.780650,0.0,0.0,0.00,0.000000,246.0,192.04,0.780650
5,2005-01-01,42.972278,Italy,15121191.0,22.0,20.58,0.935455,0.0,0.0,0.00,0.000000,22.0,20.58,0.935455
6,2005-01-01,42.972278,South Africa,15121191.0,0.1,0.34,3.400000,0.0,0.0,0.00,0.000000,0.1,0.34,3.400000
7,2005-01-01,42.972278,Spain,15121191.0,1023.7,594.26,0.580502,0.0,0.0,0.00,0.000000,1023.7,594.26,0.580502
8,2005-01-01,42.972278,Switzerland,15121191.0,0.5,1.28,2.560000,0.0,0.0,0.00,0.000000,0.5,1.28,2.560000
9,2005-01-01,42.972278,United Kingdom,15121191.0,0.0,1.49,NaN,0.0,0.0,0.00,0.000000,0.0,1.49,NaN


### Merged trade data, oil price, weather data

In [56]:
df_merged_trade_oil_weather = df_merged_trade_oil.merge(df_global_temperature, on=['TIME_PERIOD', 'PARTNER_Labels'], how='outer')
df_merged_trade_oil_weather

,TIME_PERIOD,OIL_PRICE $/bbl,PARTNER_Labels,product_x,OBS_VALUE_QUANTITY_TON_x,OBS_VALUE_VALUE_1000EURO_x,price_x,product_y,OBS_VALUE_QUANTITY_TON_y,OBS_VALUE_VALUE_1000EURO_y,price_y,OBS_VALUE_QUANTITY_TON,OBS_VALUE_VALUE_1000EURO,price,AVG_TAVG,MIN_TMIN,MAX_TMAX
0,2005-01-01,42.972278,Argentina,15121191.0,21086.3,10489.97,0.497478,0.0,0.0,0.00,0.000000,21086.3,10489.97,0.497478,NaN,NaN,NaN
1,2005-01-01,42.972278,Belgium,15121191.0,489.1,414.24,0.846943,15121990.0,1171.7,839.46,0.716446,1660.8,1253.70,0.754877,30.700000,-31.000000,79.000000
2,2005-01-01,42.972278,France,15121191.0,1295.6,886.20,0.684007,15121990.0,1631.3,1024.46,0.628002,2926.9,1910.66,0.652793,50.081198,-40.377358,124.038462
3,2005-01-01,42.972278,Germany,15121191.0,81.0,79.89,0.986296,15121990.0,1228.8,939.09,0.764233,1309.8,1018.98,0.777966,28.651592,-91.000000,129.842105
4,2005-01-01,42.972278,Ireland,15121191.0,246.0,192.04,0.780650,0.0,0.0,0.00,0.000000,246.0,192.04,0.780650,75.622811,4.285714,131.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21308,2022-08-01,NaN,Viet Nam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,287.060676,244.363636,348.666667
21309,2022-09-01,NaN,Viet Nam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,279.751352,233.600000,350.000000
21310,2022-10-01,NaN,Viet Nam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,246.000000,238.000000,260.000000
21311,2022-11-01,NaN,Viet Nam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,272.000000,252.000000,304.000000


### Merged trade data, oil price, weather, future prices data

In [57]:
df_futures_sunflower_seed['TIME_PERIOD'] = pd.to_datetime(df_futures_sunflower_seed['TIME_PERIOD'])
df_merged_trade_oil_weather_futures = df_merged_trade_oil_weather.merge(df_futures_sunflower_seed, on='TIME_PERIOD', how='outer')
df_merged_trade_oil_weather_futures.head(50)

,TIME_PERIOD,OIL_PRICE $/bbl,PARTNER_Labels,product_x,OBS_VALUE_QUANTITY_TON_x,OBS_VALUE_VALUE_1000EURO_x,price_x,product_y,OBS_VALUE_QUANTITY_TON_y,OBS_VALUE_VALUE_1000EURO_y,price_y,OBS_VALUE_QUANTITY_TON,OBS_VALUE_VALUE_1000EURO,price,AVG_TAVG,MIN_TMIN,MAX_TMAX,Sunflower Seed Price Futures
0,2005-01-01,42.972278,Argentina,15121191.0,21086.3,10489.97,0.497478,0.0,0.0,0.00,0.000000,21086.3,10489.97,0.497478,NaN,NaN,NaN,NaN
1,2005-01-01,42.972278,Belgium,15121191.0,489.1,414.24,0.846943,15121990.0,1171.7,839.46,0.716446,1660.8,1253.70,0.754877,30.700000,-31.000000,79.000000,NaN
2,2005-01-01,42.972278,France,15121191.0,1295.6,886.20,0.684007,15121990.0,1631.3,1024.46,0.628002,2926.9,1910.66,0.652793,50.081198,-40.377358,124.038462,NaN
3,2005-01-01,42.972278,Germany,15121191.0,81.0,79.89,0.986296,15121990.0,1228.8,939.09,0.764233,1309.8,1018.98,0.777966,28.651592,-91.000000,129.842105,NaN
4,2005-01-01,42.972278,Ireland,15121191.0,246.0,192.04,0.780650,0.0,0.0,0.00,0.000000,246.0,192.04,0.780650,75.622811,4.285714,131.800000,NaN
5,2005-01-01,42.972278,Italy,15121191.0,22.0,20.58,0.935455,0.0,0.0,0.00,0.000000,22.0,20.58,0.935455,80.704308,15.250000,147.888889,NaN
6,2005-01-01,42.972278,South Africa,15121191.0,0.1,0.34,3.400000,0.0,0.0,0.00,0.000000,0.1,0.34,3.400000,216.314574,144.000000,331.241379,NaN
7,2005-01-01,42.972278,Spain,15121191.0,1023.7,594.26,0.580502,0.0,0.0,0.00,0.000000,1023.7,594.26,0.580502,78.957071,-19.403226,165.015873,NaN
8,2005-01-01,42.972278,Switzerland,15121191.0,0.5,1.28,2.560000,0.0,0.0,0.00,0.000000,0.5,1.28,2.560000,-15.701613,-130.000000,87.000000,NaN
9,2005-01-01,42.972278,United Kingdom,15121191.0,0.0,1.49,NaN,0.0,0.0,0.00,0.000000,0.0,1.49,NaN,60.656665,-15.153846,116.500000,NaN


### Merged trade, oil price, weather, future prices, macroeconomic

In [58]:
df_merged = df_merged_trade_oil_weather_futures.merge(df_macroeconomic, on=['TIME_PERIOD', 'PARTNER_Labels'], how='left')
df_merged

,TIME_PERIOD,OIL_PRICE $/bbl,PARTNER_Labels,product_x,OBS_VALUE_QUANTITY_TON_x,OBS_VALUE_VALUE_1000EURO_x,price_x,product_y,OBS_VALUE_QUANTITY_TON_y,OBS_VALUE_VALUE_1000EURO_y,...,gdp_current_us,gdp_constant_2015_us,population_total,unemployment_total,agricultural_land,renewable_energy_consumption_perc_of_total,energy_use_kg_of_oil_equivalent_per_capita,fossil_fuel_energy_consumption_perc_of_total,Time Code,population_projections
0,2005-01-01,42.972278,Argentina,15121191.0,21086.3,10489.97,0.497478,0.0,0.0,0.00,...,1.987371e+11,4.352966e+11,39070501.0,11.51,1299209.0,9.64,1712.85,89.05,YR2005,39070501.0
1,2005-01-01,42.972278,Belgium,15121191.0,489.1,414.24,0.846943,15121990.0,1171.7,839.46,...,3.857148e+11,4.026519e+11,10478617.0,8.44,13830.0,2.46,5555.62,74.76,YR2005,10478617.0
2,2005-01-01,42.972278,France,15121191.0,1295.6,886.20,0.684007,15121990.0,1631.3,1024.46,...,2.196945e+12,2.224791e+12,63188395.0,8.88,293904.0,8.66,4286.54,52.30,YR2005,63188395.0
3,2005-01-01,42.972278,Germany,15121191.0,81.0,79.89,0.986296,15121990.0,1228.8,939.09,...,2.846864e+12,2.912034e+12,82469422.0,11.17,170310.0,7.28,4086.50,81.86,YR2005,82469422.0
4,2005-01-01,42.972278,Ireland,15121191.0,246.0,192.04,0.780650,0.0,0.0,0.00,...,2.118770e+11,2.077981e+11,4159914.0,4.34,43020.0,2.89,3503.64,90.92,YR2016,4755335.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21436,2022-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21437,2022-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21438,2022-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21439,2022-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
df_merged.columns

Index(['TIME_PERIOD', 'OIL_PRICE $/bbl', 'PARTNER_Labels', 'product_x',
       'OBS_VALUE_QUANTITY_TON_x', 'OBS_VALUE_VALUE_1000EURO_x', 'price_x',
       'product_y', 'OBS_VALUE_QUANTITY_TON_y', 'OBS_VALUE_VALUE_1000EURO_y',
       'price_y', 'OBS_VALUE_QUANTITY_TON', 'OBS_VALUE_VALUE_1000EURO',
       'price', 'AVG_TAVG', 'MIN_TMIN', 'MAX_TMAX',
       'Sunflower Seed Price Futures', 'country_code', 'gdp_current_us',
       'gdp_constant_2015_us', 'population_total', 'unemployment_total',
       'agricultural_land', 'renewable_energy_consumption_perc_of_total',
       'energy_use_kg_of_oil_equivalent_per_capita',
       'fossil_fuel_energy_consumption_perc_of_total', 'Time Code',
       'population_projections'],
      dtype='object')

## CLEANING EXTREME VALUES

In [60]:
import numpy as np

# Delete all inf and -inf
df_merged = df_merged.replace([np.inf, -np.inf], np.nan)

df_merged.columns

Index(['TIME_PERIOD', 'OIL_PRICE $/bbl', 'PARTNER_Labels', 'product_x',
       'OBS_VALUE_QUANTITY_TON_x', 'OBS_VALUE_VALUE_1000EURO_x', 'price_x',
       'product_y', 'OBS_VALUE_QUANTITY_TON_y', 'OBS_VALUE_VALUE_1000EURO_y',
       'price_y', 'OBS_VALUE_QUANTITY_TON', 'OBS_VALUE_VALUE_1000EURO',
       'price', 'AVG_TAVG', 'MIN_TMIN', 'MAX_TMAX',
       'Sunflower Seed Price Futures', 'country_code', 'gdp_current_us',
       'gdp_constant_2015_us', 'population_total', 'unemployment_total',
       'agricultural_land', 'renewable_energy_consumption_perc_of_total',
       'energy_use_kg_of_oil_equivalent_per_capita',
       'fossil_fuel_energy_consumption_perc_of_total', 'Time Code',
       'population_projections'],
      dtype='object')

In [61]:
df_merged['Sunflower Seed Price Futures']

0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
         ...  
21436    267.0
21437    262.0
21438    265.0
21439    267.0
21440    259.0
Name: Sunflower Seed Price Futures, Length: 21441, dtype: float64

In [63]:
# Define a function to replace extreme high values with NaN
def replace_outliers(df, column):
    z = np.abs((df[column] - df[column].mean()) / df[column].std()) # Calculate z-score for the column
    outliers = df[z > 3][column] # Identify extreme high values with a z-score greater than 3
    median = df[column].median() # Calculate the median of the column
    df.loc[z > 3, column] = np.nan # Replace extreme high values with NaN

# Define a list of columns to perform outlier treatment on
columns = ['OBS_VALUE_QUANTITY_TON', 'OBS_VALUE_VALUE_1000EURO', 'price', 'OIL_PRICE $/bbl', 'AVG_TAVG', 'MIN_TMIN', 'MAX_TMAX', 'Sunflower Seed Price Futures', 'gdp_current_us', 'population_total', 'unemployment_total', 'renewable_energy_consumption_perc_of_total', 'energy_use_kg_of_oil_equivalent_per_capita', 'fossil_fuel_energy_consumption_perc_of_total', 'population_projections']
# Iterate over the columns and call the replace_outliers function for each one
for column in columns:
    replace_outliers(df_merged, column)


df_merged

,TIME_PERIOD,OIL_PRICE $/bbl,PARTNER_Labels,product_x,OBS_VALUE_QUANTITY_TON_x,OBS_VALUE_VALUE_1000EURO_x,price_x,product_y,OBS_VALUE_QUANTITY_TON_y,OBS_VALUE_VALUE_1000EURO_y,...,gdp_current_us,gdp_constant_2015_us,population_total,unemployment_total,agricultural_land,renewable_energy_consumption_perc_of_total,energy_use_kg_of_oil_equivalent_per_capita,fossil_fuel_energy_consumption_perc_of_total,Time Code,population_projections
0,2005-01-01,42.972278,Argentina,15121191.0,21086.3,10489.97,0.497478,0.0,0.0,0.00,...,1.987371e+11,4.352966e+11,39070501.0,11.51,1299209.0,9.64,1712.85,89.05,YR2005,39070501.0
1,2005-01-01,42.972278,Belgium,15121191.0,489.1,414.24,0.846943,15121990.0,1171.7,839.46,...,3.857148e+11,4.026519e+11,10478617.0,8.44,13830.0,2.46,5555.62,74.76,YR2005,10478617.0
2,2005-01-01,42.972278,France,15121191.0,1295.6,886.20,0.684007,15121990.0,1631.3,1024.46,...,2.196945e+12,2.224791e+12,63188395.0,8.88,293904.0,8.66,4286.54,52.30,YR2005,63188395.0
3,2005-01-01,42.972278,Germany,15121191.0,81.0,79.89,0.986296,15121990.0,1228.8,939.09,...,2.846864e+12,2.912034e+12,82469422.0,11.17,170310.0,7.28,4086.50,81.86,YR2005,82469422.0
4,2005-01-01,42.972278,Ireland,15121191.0,246.0,192.04,0.780650,0.0,0.0,0.00,...,2.118770e+11,2.077981e+11,4159914.0,4.34,43020.0,2.89,3503.64,90.92,YR2016,4755335.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21436,2022-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21437,2022-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21438,2022-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21439,2022-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Define a function to replace extreme high values with NaN
def replace_outliers(df, column):
    z = np.abs((df[column] - df[column].mean()) / df[column].std()) # Calculate z-score for the column
    outliers = df[z > 3][column] # Identify extreme high values with a z-score greater than 3
    df.loc[z > 3, column] = np.nan # Replace extreme high values with NaN

# Define a list of columns to perform outlier treatment on
columns = ['price']

# Iterate over the columns and call the replace_outliers function for each one
for column in columns:
    replace_outliers(df_merged, column)



## FULL DATAFRAME (without filtering top countries)

In [ ]:
###FOR NOW
df_sunflower_total = df_merged.pivot_table(index='TIME_PERIOD', columns=['PARTNER_Labels'], values=['OBS_VALUE_QUANTITY_TON', 'OBS_VALUE_VALUE_1000EURO', 'price', 'OIL_PRICE $/bbl', 'AVG_TAVG', 'MIN_TMIN', 'MAX_TMAX', 'Corn Price Futures', 'gdp_current_us', 'population_total','unemployment_total', 'agricultural_land', 'renewable_energy_consumption_perc_of_total','energy_use_kg_of_oil_equivalent_per_capita', 'fossil_fuel_energy_consumption_perc_of_total'], margins=True)
df_sunflower_total

In [ ]:
df_sunflower_total.to_excel("sunflower_total.xlsx")

## ANALYSIS SUNFLOWER OIL IMPORT TO NL (HISTORICAL) & FILTER TOP PARTNERS

In [ ]:
df_sunflower['TIME_PERIOD'] = pd.to_datetime(df_sunflower['TIME_PERIOD'])
mask = (df_sunflower['TIME_PERIOD'] >= '2005-01-01') & (df_sunflower['TIME_PERIOD'] <= '2022-10-01')
df_filtered = df_sunflower.loc[mask]

grouped = df_filtered.groupby('PARTNER_Labels')['OBS_VALUE_QUANTITY_TON'].sum().reset_index()
grouped = grouped.sort_values(by='OBS_VALUE_QUANTITY_TON', ascending=False)
grouped = grouped.reset_index()
grouped['cumulative_sum'] = grouped['OBS_VALUE_QUANTITY_TON'].cumsum()
grouped['cumulative_sum_pct'] = grouped['cumulative_sum'] / grouped['OBS_VALUE_QUANTITY_TON'].sum()

grouped.head(10)        
        


## HANDLE MISSING DATA

In [ ]:
# Create a new DataFrame that only contains the 'price' values of Belgium, Germany and France (complete and in top-6):
df_price = df_sunflower_total[['price']].loc[:, ('price', ['Belgium', 'France', 'Germany'])]
df_price

In [ ]:
# Remove any rows that contain missing values (NaN) in the 'price' column
df_price = df_price.dropna()

In [ ]:
df_price_mean = df_price.mean(axis=1)
df_price_mean

In [ ]:
# Calculate the year-over-year percent change in 'price' for France, Germany and Belgium:
df_price_pct_change = df_price.pct_change()

# Replace NaN values with 0 (there are NaN values for the first year)
df_price_pct_change = df_price_pct_change.fillna(0)
df_price_pct_change

# Calculate the mean percent change in 'price' for each year across France and Germany and Belgium:
df_price_pct_change_mean = df_price_pct_change.mean(axis=1)
df_price_pct_change_mean


In [ ]:
# Create a new DataFrame that only contains the missing 'price' values for other countries:
df_missing_price = df_sunflower_total[['price']].loc[:, ('price', df_sunflower_total['price'].isna().any())]
df_missing_price

In [ ]:
# Fill in the missing 'price' values for each year in the new DataFrame using the mean percent change from France and Germany
df_missing_price = df_missing_price.fillna(method='ffill')
df_missing_price = df_missing_price.fillna(method='bfill')
df_missing_price = df_missing_price.fillna(df_price.iloc[-1])
df_missing_price = df_missing_price.apply(lambda x: x * (1 + df_price_pct_change_mean))

df_missing_price